# Implementation of splitting datasets by target_year

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from utils.data import (
    get_data_period,
    read_data,
    print_info,
    print_info_targets
)

In [11]:
def read_data(path, sep=None):
    if sep is not None:
        df = pd.read_csv(path, sep=sep)
    else:
        df = pd.read_csv(path)
    # display(path, df.shape, df.head(3))
    # display(df.isna().sum())
    
    return df

In [12]:
from catboost import CatBoostClassifier

import torch
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.utils import FeatureDict
from ptls.data_load.datasets import MemoryMapDataset

import torch
import torchmetrics
import pytorch_lightning as ptl

from ptls.nn import TrxEncoder, TransformerSeqEncoder, Head, RnnSeqEncoder
from ptls.frames.supervised import SeqToTargetDataset, SequenceToTarget
from ptls.frames import PtlsDataModule

from tqdm.auto import tqdm
import polars as pl

In [13]:
test_df = read_data('/home/jupyter/datasphere/project/test/test.csv')

In [14]:
test_df.shape

(633434, 41)

In [16]:
test_df.head()

,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,actv_prd_d,lst_pmnt_rcnc_d,balance,oprtn_sum_per_qrtr,oprtn_sum_per_year,frst_pmnt_date,lst_pmnt_date_per_qrtr,frst_pmnt,lst_pmnt,pmnts_sum,pmnts_nmbr,pmnts_sum_per_qrtr,pmnts_sum_per_year,pmnts_nmbr_per_qrtr,pmnts_nmbr_per_year,incm_sum,incm_per_qrtr,incm_per_year,mgd_accum_period,mgd_payment_period,phone_number,email,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt
0,0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,4543,31,464770.58,10880.44,39763.52,2004-12-31,2017-06-09,19164.90,4168.46,342134.88,151,10880.43,39763.52,3,11,122635.69,-0.00,-0.00,0.0,0.0,-1,0,1,-1,-1,398046.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,1
1,2,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,0,2209,221.24,10.54,10.54,2015-12-14,NaN,82.48,82.48,82.48,1,-0.00,-0.00,0,0,138.75,10.53,10.53,5.0,5.0,-1,-1,-1,-1,-1,0.0,МОСКВА Г,1,1,0,0
2,3,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,1615,29,27071.14,1850.63,8540.05,2011-04-13,2015-09-14,301.96,948.37,24972.87,54,1850.63,6401.21,3,10,2098.26,-0.00,2138.83,0.0,0.0,0,0,-1,-1,-1,666211.0,ИРКУТСКАЯ ОБЛ,-1,1,0,0
3,2,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,3538,30,50580.42,1219.51,6860.62,2005-12-31,2015-09-08,8455.28,406.50,31951.22,118,1219.51,4878.04,3,12,13909.69,-0.00,1982.57,4.0,4.0,-1,-1,-1,-1,-1,162610.0,ВОЛОГОДСКАЯ ОБЛ,1,1,0,1
4,0,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,276,30,7300.73,1747.83,9468.41,2010-12-09,2011-09-11,301.83,1353.97,7299.96,10,1747.83,9220.26,3,12,0.76,-0.00,248.13,0.0,0.0,-1,0,-1,-1,-1,427111.0,УДМУРТСКАЯ РЕСП,1,1,0,1


In [21]:
test_df['npo_account_id'].unique().shape

(179743,)

In [26]:
test_df['year'].min()

1993

In [ ]:
train_df = read_data('/home/jupyter/datasphere/project/test/test.csv')
trnsc_df = read_data('/home/jupyter/datasphere/project/train/trnsctns.csv', sep=';')

train_df_pl = pl.from_pandas(train_df)
trnsc_df_pl = pl.from_pandas(trnsc_df)

trx_df_pl = pl.from_pandas(trnsc_df)
trx_df_pl = trx_df_pl.with_columns(quarter = trx_df_pl['quarter'].apply(lambda x: int(x[-1])) - 1)
trx_df_dict = trx_df_pl.partition_by('npo_account_id', as_dict='True')

In [25]:
2 + 2

4

In [29]:
import datetime

def process_trx(start_trx_year=1990, start_year_targets=1993, end_year_targets=2024):
    # Данные для обучения Аггрегаты
    train_df_pl = pl.from_pandas(train_df)
    targets_train = train_df_pl.filter(
        (pl.col('year') >= start_year_targets) & (pl.col('year') < end_year_targets)
    )
    # Преобразуем год к уникальному идентификатору
    years = [i for i in range(1993, 2024)]
    year_to_id = {
        year: (i+1) for i, year in enumerate(years)
    }

    #Датафремы с нашими данными
    result = []
    t_shape = targets_train.shape[0]
    i = 0
    emtpy_date = datetime.datetime(1, 1, 1).date().strftime('%Y-%m-%d')

    empty_trx = {'npo_account_id': [0],
                              'npo_sum':[0.],
                              'npo_oprtn_date': [emtpy_date],
                              'quarter': [0], 
                              'cat_year': [0],
                              'target_year': [0],
                              #'target_churn': [0],
                              'target_quarter': [0]
                }

    for row in tqdm(targets_train.iter_rows(named=True), total=t_shape):
        #target = row['churn']
        id_user = row['npo_account_id']
        year = row['year']
        quarter = int(row['quarter'][-1]) - 1
        source_year = row['year']
        soure_q =  quarter
        ## Перейти на polars dataframe
        if id_user not in trx_df_dict:
            print('id_user not found in trx_df_dict')
            continue
        else:
            if quarter == 0:
                year -= 1
                quarter = 3
            else:
                quarter -= 1
            trx = trx_df_dict[id_user].filter(
                (pl.col('year') > start_trx_year) & (pl.col('year') <= year) & ((pl.col('year') != year) | (pl.col('quarter') < quarter))
            )
        if trx.shape[0] == 0:
            #print('loss chain')
            #display(id_user, year, quarter, trx_df_dict[id_user])
            empty_trx['npo_account_id'] = [id_user]
            empty_trx['target_quarter'] = quarter
            empty_trx['target_year'] = year_to_id[year]
            test = pl.DataFrame(empty_trx)
            test = test.with_columns(target_year = test['target_year'].cast(pl.Int32))
            #test = test.with_columns(target_churn = test['target_churn'].cast(pl.Int32))
            test = test.with_columns(target_quarter = test['target_quarter'].cast(pl.Int32))
            result.append(test)
            continue

        final_trx = trx[['npo_account_id', 'npo_sum', 'npo_oprtn_date', 'quarter']]
        final_trx = final_trx.with_columns(cat_year = trx['year'].map_dict(year_to_id))
        final_trx = final_trx.with_columns(target_year = source_year)
        #final_trx = final_trx.with_columns(target_churn = target)
        final_trx = final_trx.with_columns(target_quarter = soure_q)

        result.append(final_trx)
        
    return result

In [ ]:
test_res = process_trx()

100%|██████████| 633434/633434 [46:27<00:00, 227.26it/s] 


In [ ]:
len(test_res)

633434

In [ ]:
test_res = pl.concat(test_res)

In [ ]:
test_res

npo_account_id,npo_sum,npo_oprtn_date,quarter,cat_year,target_year,target_quarter
str,f64,str,i64,i64,i32,i32
"""0xAA9E8DDF1310…",19164.9,"""2004-12-31""",3,12,2017,1
"""0xAA9E8DDF1310…",5937.83,"""2004-12-31""",3,12,2017,1
"""0xAA9E8DDF1310…",617.07,"""2005-01-10""",0,13,2017,1
"""0xAA9E8DDF1310…",614.26,"""2005-02-08""",0,13,2017,1
"""0xAA9E8DDF1310…",617.07,"""2005-03-09""",0,13,2017,1
"""0xAA9E8DDF1310…",255.78,"""2005-04-07""",1,13,2017,1
"""0xAA9E8DDF1310…",583.74,"""2005-05-06""",1,13,2017,1
"""0xAA9E8DDF1310…",617.07,"""2005-06-07""",1,13,2017,1
"""0xAA9E8DDF1310…",1084.26,"""2005-06-30""",1,13,2017,1


In [34]:
#test_res = pl.concat(test_res)
test_res.write_csv('test_answer.csv')

In [ ]:
len(test_res)

95993917